In [1]:
from secret_key import openai_api_key
import os
os.environ["OPENAI_API_KEY"]=openai_api_key

In [6]:
from langchain.llms import OpenAI

In [7]:
llm=OpenAI(temperature=0.5)

In [8]:
llm("Who is the father of AI?")

'\n\nJohn McCarthy is widely credited as the "father of Artificial Intelligence" for his work in the 1950s and 1960s.'

### Prompt Template

In [9]:
from langchain.prompts import PromptTemplate

In [11]:
from langchain.chains import LLMChain

In [40]:
name_template=PromptTemplate(
    input_variables=['cuisine', 'type', 'place'],
    template='''
    I am planning to open a restaurant. I'll provide you with the details below, Suggest a fancy and catchy name for it. (Give the name with a relevant emoji)

    Type: {type}
    Cuisine: {cuisine}
    Location: {place}
    '''
)

In [41]:
name_chain=LLMChain(llm=llm,prompt=name_template,output_key="restaurant_name")

In [45]:
menu_template=PromptTemplate(
    input_variables=['restaurant_name','von','menu'],
    template="Give me a comma separated values of menu items for the {von} restaurant named {restaurant_name} which serves {menu}"
)

In [46]:
menu_chain=LLMChain(llm=llm,prompt=menu_template,output_key="menu_items")

In [48]:
price_template=PromptTemplate(
    input_variables=['menu_items','location'],
    template='''
    Give me a comma separated price list for the below mentioned menu items.
    
    {menu_items}
    
    The restaurant is located in {location}. Assign the price accordingly. Format the output as a comma separated values.
    '''
)

In [49]:
price_chain=LLMChain(llm=llm,prompt=price_template,output_key="price")

In [50]:
from langchain.chains import SequentialChain
from langchain.memory import SimpleMemory

In [62]:
def generate_name_and_menu(res_type,cuisine,place):
    overall_chain = SequentialChain(
        memory=SimpleMemory(memories={"menu": cuisine, "location":place, "von":res_type}),
        chains=[name_chain, menu_chain, price_chain],
        input_variables=["cuisine", "type","place"],
        output_variables=["restaurant_name","menu_items","price"]
    )
    
    output=overall_chain({"cuisine":cuisine,"type":res_type,"place":place})
    print(output)

In [ ]:
generate_name_and_menu("Vegetarian","South India","Coimbatore")